[View in Colaboratory](https://colab.research.google.com/gist/alextp/9568ab40f6ed6f9a3ba4736f6aef6127/debugging-graph-first-models-with-eager-execution.ipynb)

This colab uses eager execution and the Python debugger to modify the execution of a translation model. This combination lets you quickly explore counterfactuals when researching and designing modifications to a model.

The model, Transformer from [Tensor2Tensor](https://github.com/tensorflow/tensor2tensor), was originally written with graph building in mind. Executing it eagerly can still be helpful!

In [0]:
#@title License (double click to show)
# Copyright 2018 The TensorFlow Authors.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import tensorflow as tf
import pdb
tfe = tf.contrib.eager

tf.enable_eager_execution()

In [2]:
!pip install -q -U tensor2tensor
from tensor2tensor.models import transformer

In [3]:
#@title Create a tensor2tensor translation model, fetch a checkpoint (double click to show)
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry

import numpy as np
import os

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

# Fetch the problem
ende_problem = problems.problem("translate_ende_wmt32k")

# Copy the vocab file locally so we can encode inputs and decode model outputs
# All vocabs are stored on GCS
vocab_name = "vocab.ende.32768"
vocab_file = os.path.join(gs_data_dir, vocab_name)
!gsutil cp {vocab_file} {data_dir}

# Get the encoders from the problem
encoders = ende_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

# Copy the pretrained checkpoint locally
ckpt_name = "transformer_ende_test"
gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
!gsutil -q cp -R {gs_ckpt} {checkpoint_dir}
checkpoint_path = tf.train.latest_checkpoint(
    os.path.join(checkpoint_dir, ckpt_name))

# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_base"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="translate_ende_wmt32k")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, tf.estimator.ModeKeys.EVAL)

Copying gs://tensor2tensor-data/vocab.ende.32768...
/ [1 files][316.4 KiB/316.4 KiB]                                                
Operation completed over 1 objects/316.4 KiB.                                    
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0


We've created a Transformer model and fetched an existing training checkpoint. It hasn't created variables yet, and we want to load them from the checkpoint before they're used (restore-on-create) so the first run of the model outputs the correct value. The `tfe.restore_variables_on_create` API looks up variables by name on creation and restores their values.

In [4]:
with tfe.restore_variables_on_create(checkpoint_path):
  model_output = translate_model.infer(encode("Eager execution"))
print(decode(model_output["outputs"]))

INFO:tensorflow:Greedy Decoding
Hinrichtung


Using global variable names can get somewhat fragile, so for new code we recommend the object-based `tf.keras.Model.save_weights` or `tf.train.Checkpoint`. However, these require some small code changes to work with existing graph building code.

The Transformer model translates "Eager execution" in English to "Hinrichtung" in German, which refers to capital punishment rather than getting things done. Transformer first encodes the English, then decodes to German. We'll add a debugging hook at the start of the decode phase (once the encodings have been finalized) and see if we can correct the translation.

In [0]:
previous_fast_decode = transformer.fast_decode
def debug_fn(*args, **kwargs):
  pdb.set_trace()
  return previous_fast_decode(*args, **kwargs)  # "step" in pdb to step in
transformer.fast_decode = debug_fn  # Add our debugging hook to Transformer

Now that we've "monkey patched" the model, we'll drop into a debugger just before decoding starts. In most cases it'd be simpler to add the `pdb.set_trace()` call to the code directly, but in this case we're working with prepackaged library code.

First, let's find an encoding which represents the correct sense of "execution". Then we'll patch part of that encoding into the encoding of "Eager execution" to fix the translation. Feel free to poke around with the debugger (e.g. print a Tensor's value), but your main task is to save the encodings by assigning them to an attribute of the function:

```
(running the next cell drops you into a pdb shell)
step
fast_decode.previous_encoding = encoder_output
continue

```

You can type `next` (or `n`) a few times before `continue` to watch the decoding ops run.

In [7]:
model_output = translate_model.infer(encode("Immediate running"))
print(decode(model_output["outputs"]))

> <ipython-input-6-ee9b4225ba2a>(4)debug_fn()
-> return previous_fast_decode(*args, **kwargs)  # "step" in pdb to step in
(Pdb) step
--Call--
> /usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py(427)fast_decode()
-> def fast_decode(encoder_output,
(Pdb) fast_decode.previous_encoding = encoder_output
(Pdb) continue
Sofortige Durchführung


Now we have an encoding saved which gets the correct sense for "execution".

In [8]:
# Assumes you followed the pdb instructions above!
transformer.fast_decode.previous_encoding

<tf.Tensor: id=9528, shape=(1, 4, 512), dtype=float32, numpy=
array([[[-0.15239455,  0.12273102, -0.11209048, ..., -0.12478986,
          0.37216735, -0.40987235],
        [-0.2686283 ,  0.51448774,  0.03650613, ...,  0.08731575,
          0.51110077, -0.6646815 ],
        [-0.24441548,  0.36622533,  0.11685672, ...,  0.21941349,
         -0.03304008, -0.579611  ],
        [-0.03339856, -0.01185844,  0.00579634, ...,  0.00294734,
          0.00136655, -0.01362935]]], dtype=float32)>

Let's replace part of the encoding for "Eager execution" with the encoding of "Immediate running".

Again we'll drop into a pdb shell. This time we'll run some TensorFlow operations to patch the encodings while the model is running.

```
(running the next cell again drops you into a pdb shell)
step
encoder_output = tf.concat([fast_decode.previous_encoding[:, :3], encoder_output[:, 3:]], axis=1)
continue
```

In [9]:
model_output = translate_model.infer(encode("Eager execution"))
print(decode(model_output["outputs"]))

> <ipython-input-6-ee9b4225ba2a>(4)debug_fn()
-> return previous_fast_decode(*args, **kwargs)  # "step" in pdb to step in
(Pdb) step
--Call--
> /usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py(427)fast_decode()
-> def fast_decode(encoder_output,
(Pdb) encoder_output = tf.concat([fast_decode.previous_encoding[:, :3], encoder_output[:, 3:]], axis=1)
(Pdb) continue
sofortige Ausführung


We get a different decoding, with the correct sense of "execution". Likely we're keeping just the encoding of "tion" from "Eager execution", so no great breakthrough in translation modeling.

Similarly it's possible to modify attention vectors, or change words during decoding to help debug a beam search.

This colab was adapted from the [Tensor2Tensor colab](https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb). Credit to Ankur Taly for its concept.